In [2]:
from Binaries import *

In [8]:
for index in [-1]:

    Buffer = RandomTrace("nuria", index)

    print(f"{index}/{Buffer.all_n_files} {Buffer.random_file}: {len(Buffer._these_traces)} traces" )

    station = np.array(Buffer._these_traces)
    pmt1, pmt2, pmt3 = station[:,0], station[:,1], station[:,2]
    I_vem = np.array([120. for i in range(3)])

    while True:

        pmt_has_converged = 0
        n_triggers = np.zeros(3)

        for pmt1, pmt2, pmt3 in zip(station[:,0], station[:,1], station[:,2]):

            for bins in zip(pmt1, pmt2, pmt3):

                # assume all PMTs are working, k70 = 1.75
                # check the actual trigger rate
                if np.all(bins >= 1.75 * I_vem):

                    for i in range(3):
                        if bins[i] >= 2.5 * I_vem[i]: n_triggers[i] += 1

        trigger_rates = n_triggers / (len(Buffer._these_traces) * 2048 * 8.33e-8)

        print(f"VEM_peak estimate: {I_vem} ===> {trigger_rates} Hz")
        
        for i in range(3):
            if trigger_rates[i] <= 60: I_vem[i] -= 10
            elif 60 < trigger_rates[i] <= 68: I_vem[i] -= 1
            elif 68 < trigger_rates[i] <= 72: pmt_has_converged += 1
            elif 72 < trigger_rates[i] <= 80: I_vem[i] += 1
            elif 80 < trigger_rates[i]: I_vem[i] += 10

        if pmt_has_converged >= 3: break

-1/800 nuria_randoms0800.csv: 5000 traces
VEM_peak estimate: [120. 120. 120.] ===> [35.17031813 43.37672569 37.515006  ] Hz
VEM_peak estimate: [110. 110. 110.] ===> [63.30657263 69.16829232 62.13422869] Hz
VEM_peak estimate: [109. 110. 109.] ===> [65.6512605  69.16829232 63.30657263] Hz
VEM_peak estimate: [108. 110. 108.] ===> [67.99594838 71.51298019 65.6512605 ] Hz
VEM_peak estimate: [107. 110. 107.] ===> [72.68532413 71.51298019 69.16829232] Hz
VEM_peak estimate: [108. 110. 107.] ===> [69.16829232 71.51298019 69.16829232] Hz


In [7]:
n_Th_t2 = 0

Trigger = HardwareClassifier()

for n, (pmt1, pmt2, pmt3) in enumerate(zip(station[:,0], station[:,1], station[:,2])):

    trace = np.zeros((3,2048))

    trace[0] = np.array(pmt1) / I_vem[0]
    trace[1] = np.array(pmt2) / I_vem[1]
    trace[2] = np.array(pmt3) / I_vem[2]

    if Trigger.Th(3.2, trace):
        n_Th_t2 += 1

print(f"Th2 trigger rate: {n_Th_t2 / (2048 * 8.33e-9 * n)} Hz")

Th2 trigger rate: 58.62892266328415 Hz
